In [2]:
import geopandas as gpd
import importlib
import json
import lxml
import math
import matplotlib
import numpy as np
import os
import pandas as pd
import pyproj as proj
import rasterio
import requests
import seaborn as sns
import sys
import zipfile
import fiona


In [119]:
reef_name = 'US_Virgin_Islands'
base_dir = '/Users/bonnieludka/Spaceship/IceSAT2/bathymetry_python_20210113'
data_dir = os.path.join(base_dir,'data')
reef_path = os.path.join(data_dir,reef_name)
gt_dir = os.path.join(reef_path,'Ground_truth','USGS_EAARL_2014_med')

filenames = [f for f in os.listdir(gt_dir) if f.endswith('.asc') if not f.startswith('.')]
print(filenames)

z_all = np.array([])
lat_all = np.array([])
lon_all = np.array([])
for X in range(0,7):
    print(X)
    fpath = os.path.join(reef_path,'Ground_truth','USGS_EAARL_2014_med',filenames[X])
    data_info= np.loadtxt(fpath, dtype = 'str',max_rows=6)
    ncols = int(data_info[0,1])
    nrows = int(data_info[1,1])
    xllcorner = float(data_info[2,1])
    yllcorner = float(data_info[3,1])
    cellsize = float(data_info[4,1])
    NODATA_value = float(data_info[5,1]) #does it matter if this is an int or float?
    
    zmflipped = np.loadtxt(fpath, skiprows=6)
    zm = np.flipud(zmflipped)
    
    NX    = ncols-1
    NY    = nrows-1
    lon1  = xllcorner
    lat1  = yllcorner
    delta = cellsize 
    
    # Create longitude and latitude coords
    lonc = np.arange(lon1,delta*NX+lon1+delta/2,delta) 
    latc = np.arange(lat1,delta*NY+lat1+delta/2,delta)
    
    # create lat and lon matrix that matches z elevation matrix
    lonm, latm = np.meshgrid(lonc, latc, sparse=False, indexing='xy')
    
    # unravel matrices into vectors
    zvr = np.ravel(zm)
    lonvr = np.ravel(lonm)
    latvr = np.ravel(latm)
    
    # remove bad data
    zv = np.delete(zvr, np.where(zvr == -9999.))
    lonv = np.delete(lonvr, np.where(zvr == -9999.))
    latv = np.delete(latvr, np.where(zvr == -9999.))
    
    # combine data
    z_all = np.hstack((z_all,zv))
    lat_all = np.hstack((lat_all,latv))
    lon_all = np.hstack((lon_all,lonv))
    print(z_all.shape)
    print(lat_all.shape)
    print(lon_all.shape)
    

['USGS_EAARL_2014_med_001_001.asc', 'USGS_EAARL_2014_med_001_002.asc', 'USGS_EAARL_2014_med_001_003.asc', 'USGS_EAARL_2014_med_001_004.asc', 'USGS_EAARL_2014_med_002_001.asc', 'USGS_EAARL_2014_med_002_002.asc', 'USGS_EAARL_2014_med_002_003.asc']
0
1
2
3
4
5
6


In [120]:
# put data into dataframe
gt_data = [z_all,lat_all,lon_all]
df = pd.DataFrame(gt_data).T
df.columns = ['Elevation','Latitude','Longitude']
print(df)

# save to .csv file
save_path = os.path.join(gt_dir,'USGS_EAARL_2014_med_all.csv')
print(save_path)
df.to_csv(save_path)

          Elevation   Latitude  Longitude
0        -76.830002  18.274343 -65.032909
1        -76.860001  18.274343 -65.032873
2        -77.110001  18.274343 -65.032855
3        -77.040001  18.274343 -65.032837
4        -76.980003  18.274343 -65.032783
...             ...        ...        ...
36934569        NaN  18.352402 -64.880080
36934570        NaN  18.352402 -64.880062
36934571        NaN  18.352402 -64.880044
36934572        NaN  18.352402 -64.880026
36934573        NaN  18.352402 -64.880008

[36934574 rows x 3 columns]
/Users/bonnieludka/Spaceship/IceSAT2/bathymetry_python_20210113/data/US_Virgin_Islands/Ground_truth/USGS_EAARL_2014_med/USGS_EAARL_2014_med_all


In [129]:
# Create longitude and latitude coords
lonc   = np.arange(lon1,delta*NX+lon1+delta/2,delta) 
latc   = np.arange(lat1,delta*NY+lat1+delta/2,delta)

# create lat and lon matrix that matches z elevation matrix
lonm, latm = np.meshgrid(lonc, latc, sparse=False, indexing='xy')

# unravel matrices into vectors
zvr = np.ravel(zm)
lonvr = np.ravel(lonm)
latvr = np.ravel(latm)

# remove bad data
zv = np.delete(zvr, np.where(zvr == -9999.))
lonv = np.delete(lonvr, np.where(zvr == -9999.))
latv = np.delete(latvr, np.where(zvr == -9999.))
    
print(latv.shape)
print(lonv.shape)
print(zv.shape)



(603,)
(603,)
(603,)
